## Imports

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import asyncio
import nest_asyncio
from shazamio import Shazam
import json
import requests

## Record Audio

Record a 10 second audio clip and save it to later use shazam to identify the song.

In [ ]:
fs = 44100  # Sample rate
seconds = 10  # Duration of recording

myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
sd.wait()  # Wait until recording is finished
write('output.wav', fs, myrecording)  # Save as WAV file 

## Identify the Song using Shazam

Load the audio clip and use the Shazam library to identify the song. Then save the result to a file.

In [8]:
# Allow running nested event loops in Jupyter notebooks or IPython shells
nest_asyncio.apply()

async def main():
    shazam = Shazam()
    out = await shazam.recognize('output.wav')  # rust version, use this!
    # save the output as a json file
    with open('output.json', 'w') as f:
        json.dump(out, f)
    print(out)

# Create a new event loop
loop = asyncio.new_event_loop()
# Set the new event loop as the current one
asyncio.set_event_loop(loop)
# Run the coroutine in the new event loop
loop.run_until_complete(main())

# Close the event loop
loop.close()

out = json.load(open('output.json'))

{'matches': [{'id': '657775429', 'offset': 52.053070312, 'timeskew': 7.355213e-05, 'frequencyskew': 0.00076544285}], 'location': {'accuracy': 0.01}, 'timestamp': 1738039424, 'timezone': 'Europe/Moscow', 'track': {'layout': '5', 'type': 'MUSIC', 'key': '657775429', 'title': 'Doppelgänger (feat. Druwu)', 'subtitle': 'Chäckra, Lëri & Seyn', 'images': {'background': 'https://is1-ssl.mzstatic.com/image/thumb/Music126/v4/d0/9f/37/d09f37b7-2058-f08a-4210-020981ab63a0/artwork.jpg/400x400cc.jpg', 'coverart': 'https://is1-ssl.mzstatic.com/image/thumb/Music126/v4/d0/9f/37/d09f37b7-2058-f08a-4210-020981ab63a0/artwork.jpg/400x400cc.jpg', 'coverarthq': 'https://is1-ssl.mzstatic.com/image/thumb/Music126/v4/d0/9f/37/d09f37b7-2058-f08a-4210-020981ab63a0/artwork.jpg/400x400cc.jpg', 'joecolor': 'b:001219p:d5b8c8s:cbabd5t:aa97a5q:a28daf'}, 'share': {'subject': 'Doppelgänger (feat. Druwu) - Chäckra, Lëri & Seyn', 'text': 'Doppelgänger (feat. Druwu) by Chäckra, Lëri & Seyn', 'href': 'https://www.shazam.com/

## Spotify API for Song Features

Once we have the song name and artisrt, we can use the Spotify API to get the song features.

### Spotify API Access

In [2]:
CLIENT_ID = '109b462b7b2849899953426435df8169'
CLIENT_SECRET = 'a929ffb635444a22aca384b6dc47fc16'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

### Fetching the Song Features

In [22]:
# get the features of a song based given the song title and artist

def find_song(title, artist):
    # search for the song
    r = requests.get('https://api.spotify.com/v1/search', headers=headers, params={
        'q': title,
        'type': 'track'
    })

    # convert the response to JSON
    r = r.json()

    # get the first song from the results
    song = r['tracks']['items'][0]

    # return the song
    return song

def get_features(title, artist):
    # get the song
    song = find_song(title, artist)

    # get the song's features
    r = requests.get('https://api.spotify.com/v1/audio-features/' + song['id'], headers=headers)

    # convert the response to JSON
    r = r.json()

    # return the features
    return r

# get the title and artist of the song
title = out['track']['title']
artist = out['track']['subtitle']

# get the features of the song
features = get_features(title, artist)

# get only the numerical features
numerical_features = {k: features[k] for k in features if type(features[k]) in [int, float]}
print(numerical_features)

{'danceability': 0.276, 'energy': 0.154, 'key': 8, 'loudness': -18.451, 'mode': 0, 'speechiness': 0.0327, 'acousticness': 0.968, 'instrumentalness': 0.363, 'liveness': 0.109, 'valence': 0.144, 'tempo': 80.763, 'duration_ms': 209815, 'time_signature': 3}
